In [2]:
from transformers import TFCamembertModel,  CamembertConfig, CamembertTokenizerFast
# Then what you need from tensorflow.keras
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
# And pandas for data import + sklearn because you allways need sklearn
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
def toStr(x):
    if pd.isna(x):
        return ""
    else:
        x = str(x).replace("\n", " ").strip()
        return x

In [4]:
df = pd.read_csv("../data/aids_mt.csv", sep="\t")

In [5]:
from bs4 import BeautifulSoup
df["name"] = df["name"].apply(lambda x: toStr(x))
df["aidDetails"] = df["aidDetails"].apply(lambda x: BeautifulSoup(toStr(x)).get_text())
df["eligibility"] = df["eligibility"].apply(lambda x: BeautifulSoup(toStr(x)).get_text())

In [6]:
ok_cols = ["name", "aidDetails", "eligibility", "classes"]

df = df[ok_cols]

In [7]:
import ast 

df = df.reset_index(drop=True)
for i in range(df.shape[0]):
    for k in range(12):
        tmp = df.loc[i,"classes"].replace("[", "").replace("]", "").split(" ")
        tmp = [int(elt)-1 for elt in tmp if elt != ""]
        if k in tmp:
            df.loc[i,k] = 1
        else:
            df.loc[i,k] = 0
            

In [8]:
df = df.drop("classes", axis=1)

In [10]:
df["list"] = df[[0,1,2,3,4,5,6,7,8,9,10,11]].apply(lambda x: [e for e in x], axis=1)

In [13]:
df

,name,aidDetails,eligibility,0,1,2,3,4,5,6,7,8,9,10,11,list
0,Financer les études et tests préalables aux in...,Aide - « Études et tests préalables aux in...,Critères d'éligibilité Pertinence du ...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,Soutenir financièrement le développement de l'...,Le dispositif de l'ADEME – ORPLAST (Objectif...,"Sont éligibles, les dossiers portant sur u...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,Financer les études de préfiguration d'une dém...,Aide - « Étude de préfiguration d'une démarc...,,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,Réaliser des investissements de décarbonation ...,Plusieurs dispositifs sont ouverts aux entre...,"Pour que la demande d'aide soit éligible, ...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,Financer des audits énergétiques en entreprise...,Aide - « Financement des audits énergétiqu...,Critères d'éligibilité : u ne étude de fais...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,Soutenir l'émergence de solutions innovantes p...,La Région soutient l'émergence de solution...,,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
386,Sortir de vacance les logements locatifs privés,Le présent dispositif vise à remettre sur le...,Bénéficiaires éligibles : Proprié...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
387,Soutenir l'économie de proximité en finançant ...,Aide - « Fonds régional des territoires (F...,,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
388,Soutenir le développement d'investissements ag...,La Région Occitanie est aux côtés des acteur...,Bénéficiaires Exploitations agricole...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."


In [15]:
df[["text", "list"]].to_csv("../data/aids_mt_preprocessed.csv", sep="\t", index=False)

In [8]:
from transformers import CamembertForSequenceClassification, CamembertConfig, CamembertTokenizerFast

model = CamembertForSequenceClassification.from_pretrained("camembert-base", num_labels=12)
tokenizer = CamembertTokenizerFast.from_pretrained("camembert-base")
model = model.to("cuda")

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.weight'

In [14]:
max_length = 512
df["text"] = df[["name", "aidDetails"]].apply(lambda x: ". ".join(x), axis=1)

In [10]:
df = df[["text", 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]]

In [11]:
df_train, df_test = train_test_split(df, test_size=0.05, random_state=42)

In [12]:
train_texts = df_train.text.tolist()
train_labels = df_train.drop("text", axis=1).values
test_texts = df_test.text.tolist()
test_labels = df_test.drop("text", axis=1).values


In [13]:
train_labels = train_labels.reshape((-1,12))
test_labels = test_labels.reshape((-1,12))


In [14]:
import torch

In [15]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_length, return_tensors="pt")
valid_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=max_length, return_tensors="pt")

for k in train_encodings.keys():
    train_encodings[k] = train_encodings[k].int().clone().detach()
for k in valid_encodings.keys():
    valid_encodings[k] = valid_encodings[k].int().clone().detach()

In [16]:
import torch
from torch.utils.data import Dataset
import numpy as np 

class AidsDataset(Dataset):
    def __init__(self, encodings, labels, max_len=512):
        self.labels = np.array(labels)
        self.encodings = encodings
        self.max_len = max_len
        
        
    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [17]:
train_dataset = AidsDataset(train_encodings, train_labels)
valid_dataset = AidsDataset(valid_encodings, test_labels)

In [18]:
from sklearn.metrics import accuracy_score, f1_score
from torch import nn

def compute_metrics(pred):
  labels = pred.label_ids
  preds = np.array(nn.Sigmoid()(torch.Tensor(pred.predictions)).detach().cpu().numpy() >= 0.5, dtype=float)
  print("preds: ", preds)
  print("labels: ", labels)
  # calculate accuracy using sklearn's function
  acc = accuracy_score(labels, preds)
  f1 = f1_score(labels, preds, average="macro")
  return {
      'accuracy': acc,
      'f1 score': f1
  }

In [19]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=2,  # batch size per device during training
    per_device_eval_batch_size=10,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
    logging_steps=100,               # log & save weights each logging_steps
    evaluation_strategy="steps",     # evaluate each `logging_steps`
)

In [20]:
from torch import nn

class MultilabelTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get('logits')
        loss_fct = nn.BCEWithLogitsLoss()
        loss = loss_fct(logits.view(-1, self.model.config.num_labels),
                        labels.float().view(-1, self.model.config.num_labels))
        return (loss, outputs) if return_outputs else loss

In [21]:
trainer = MultilabelTrainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=valid_dataset,          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)

In [22]:
# train the model
trainer.train()

***** Running training *****
  Num examples = 370
  Num Epochs = 3
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 555
  0%|          | 0/555 [00:00<?, ?it/s]C:\Users\amine\anaconda3\envs\thesis\lib\site-packages\ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
 18%|█▊        | 100/555 [00:31<02:15,  3.36it/s]***** Running Evaluation *****
  Num examples = 20
  Batch size = 10


{'loss': 0.6566, 'learning_rate': 1e-05, 'epoch': 0.54}


C:\Users\amine\anaconda3\envs\thesis\lib\site-packages\sklearn\metrics\_classification.py:1570: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))

 18%|█▊        | 100/555 [00:32<02:15,  3.36it/s]C:\Users\amine\anaconda3\envs\thesis\lib\site-packages\ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]


preds:  [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
labels:  [[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0.]
 [0. 0.

 36%|███▌      | 200/555 [01:02<01:46,  3.34it/s]***** Running Evaluation *****
  Num examples = 20
  Batch size = 10


...

C:\Users\amine\anaconda3\envs\thesis\lib\site-packages\sklearn\metrics\_classification.py:1570: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))

 90%|█████████ | 500/555 [03:16<01:04,  1.17s/it]Saving model checkpoint to ./results\checkpoint-500
Configuration saved in ./results\checkpoint-500\config.json


preds:  [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
labels:  [[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0.]
 [0. 0.

Model weights saved in ./results\checkpoint-500\pytorch_model.bin
C:\Users\amine\anaconda3\envs\thesis\lib\site-packages\ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
100%|██████████| 555/555 [04:24<00:00,  1.19s/it]

Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./results\checkpoint-500 (score: 0.3893605172634125).
100%|██████████| 555/555 [04:24<00:00,  2.10it/s]

{'train_runtime': 264.6535, 'train_samples_per_second': 4.194, 'train_steps_per_second': 2.097, 'train_loss': 0.44381331710128097, 'epoch': 3.0}


TrainOutput(global_step=555, training_loss=0.44381331710128097, metrics={'train_runtime': 264.6535, 'train_samples_per_second': 4.194, 'train_steps_per_second': 2.097, 'train_loss': 0.44381331710128097, 'epoch': 3.0})

In [23]:
inpt = tokenizer.encode_plus(df.loc[0,"text"], add_special_tokens=True, max_length=512, return_tensors="pt")
for k in inpt.keys():
    inpt[k] = inpt[k].int().clone().to("cuda")
out = model(**inpt)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [25]:
out.logits

tensor([[-2.2787, -2.6698, -1.5949, -2.1019, -1.0884, -2.4630, -1.3397, -1.9340,
         -2.2757, -1.4232, -3.0292, -2.9219]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

In [26]:
nn.Sigmoid()(out.logits)

tensor([[0.0929, 0.0648, 0.1687, 0.1089, 0.2519, 0.0785, 0.2076, 0.1263, 0.0932,
         0.1942, 0.0461, 0.0511]], device='cuda:0', grad_fn=<SigmoidBackward0>)

In [27]:
df.loc[0]

text    Financer les études et tests préalables aux in...
0                                                     1.0
1                                                     0.0
2                                                     0.0
3                                                     0.0
4                                                     0.0
5                                                     0.0
6                                                     0.0
7                                                     0.0
8                                                     0.0
9                                                     0.0
10                                                    0.0
11                                                    0.0
Name: 0, dtype: object

In [1]:
df


NameError: name 'df' is not defined